In [ ]:
!pip install google-cloud-logging
import logging
import google.cloud.logging
import google.cloud.logging_v2 as logging_v2
from os import environ

client = logging_v2.client.Client()

# set the format for the log
google_log_format= logging.Formatter(
fmt='%(name)s | %(module)s | %(funcName)s | %(message)s',
datefmt='%Y-%m-$dT%H:%M:%S')


handler = client.get_default_handler()
handler.setFormatter(google_log_format)

cloud_logger = logging.getLogger("vertex-ai-notebook-logger")
cloud_logger.setLevel("CRITICAL")
cloud_logger.addHandler(handler)

log = logging.getLogger("vertex-ai-notebook-logger")
log.critical("This is a log from a Vertex AI Notebook!")

import os
os.environ['OMP_NUM_THREADS'] = "1"
os.environ['MKL_NUM_THREADS'] = "1"
os.environ['OPENBLAS_NUM_THREADS'] = "1"

In [ ]:
import pandas as pd
import numpy as np
import pywt
from tsfresh import extract_features
from tsfresh.utilities.distribution import MultiprocessingDistributor
from tqdm import tqdm
from tsfresh import extract_relevant_features
import tsfresh

path = 'gcs/hms_applied_cv/'
offset = 1
path_for_file = path + f'hms-harmful-brain-activity-classification/new_features{offset}.csv'